# F1 Report System

Two-agent system powered by Gemini 1.5 Flash:
- Agent 1: Data Collection (validates input, retrieves race data via FastF1)
- Agent 2: Report Generation (creates social media content from collected data)

## 1. Install Dependencies

In [7]:
# Install required packages
%pip install -q google-cloud-aiplatform
%pip install -q fastf1
%pip install -q pandas
%pip install -q python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2. Import Libraries

In [1]:
# Standard library imports
import os
import json
from datetime import datetime
from typing import Dict, List, Optional, Any

# Third-party imports
import pandas as pd
from dotenv import load_dotenv

# FastF1 for F1 data retrieval
import fastf1

# Google Cloud Vertex AI for Gemini
import vertexai
from vertexai.generative_models import GenerativeModel, Content, Part

print("Libraries imported successfully")

Libraries imported successfully


## 3. Configuration

In [2]:
# Load environment variables
load_dotenv()

# Google Cloud configuration
PROJECT_ID = os.getenv('GCP_PROJECT_ID', 'gen-lang-client-0467867580')
LOCATION = os.getenv('GCP_LOCATION', 'us-central1')
MODEL_NAME = 'gemini-1.5-flash'

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)
print(f"Vertex AI initialized for project: {PROJECT_ID}")

# Enable FastF1 cache
cache_dir = 'f1_cache'
os.makedirs(cache_dir, exist_ok=True)
fastf1.Cache.enable_cache(cache_dir)
print("FastF1 cache enabled")

Vertex AI initialized for project: gen-lang-client-0467867580
FastF1 cache enabled


## 4. Session Memory

In [3]:
# In-memory session storage
session_memory = {
    "reports": {},
    "query_history": []
}

def store_report(race_id: str, report_data: Dict[str, Any]) -> None:
    """Store a generated report in session memory."""
    session_memory["reports"][race_id] = {
        "data": report_data,
        "timestamp": datetime.now().isoformat()
    }
    session_memory["query_history"].append({
        "race_id": race_id,
        "timestamp": datetime.now().isoformat()
    })

def get_report(race_id: str) -> Optional[Dict[str, Any]]:
    """Retrieve a report from session memory."""
    return session_memory["reports"].get(race_id)

def get_history() -> List[Dict[str, Any]]:
    """Get the query history."""
    return session_memory["query_history"]

print("Session memory initialized")

Session memory initialized


## 5. F1 2025 Calendar

In [4]:
# F1 2025 Calendar for validation
F1_2025_CALENDAR = {
    1: {"name": "Bahrain Grand Prix", "circuit": "Bahrain International Circuit"},
    2: {"name": "Saudi Arabian Grand Prix", "circuit": "Jeddah Corniche Circuit"},
    3: {"name": "Australian Grand Prix", "circuit": "Albert Park Circuit"},
    4: {"name": "Japanese Grand Prix", "circuit": "Suzuka International Racing Course"},
    5: {"name": "Chinese Grand Prix", "circuit": "Shanghai International Circuit"},
    6: {"name": "Miami Grand Prix", "circuit": "Miami International Autodrome"},
    7: {"name": "Emilia Romagna Grand Prix", "circuit": "Autodromo Enzo e Dino Ferrari"},
    8: {"name": "Monaco Grand Prix", "circuit": "Circuit de Monaco"},
    9: {"name": "Spanish Grand Prix", "circuit": "Circuit de Barcelona-Catalunya"},
    10: {"name": "Canadian Grand Prix", "circuit": "Circuit Gilles Villeneuve"},
    11: {"name": "Austrian Grand Prix", "circuit": "Red Bull Ring"},
    12: {"name": "British Grand Prix", "circuit": "Silverstone Circuit"},
    13: {"name": "Belgian Grand Prix", "circuit": "Circuit de Spa-Francorchamps"},
    14: {"name": "Hungarian Grand Prix", "circuit": "Hungaroring"},
    15: {"name": "Dutch Grand Prix", "circuit": "Circuit Zandvoort"},
    16: {"name": "Italian Grand Prix", "circuit": "Autodromo Nazionale di Monza"},
    17: {"name": "Azerbaijan Grand Prix", "circuit": "Baku City Circuit"},
    18: {"name": "Singapore Grand Prix", "circuit": "Marina Bay Street Circuit"},
    19: {"name": "United States Grand Prix", "circuit": "Circuit of the Americas"},
    20: {"name": "Mexico City Grand Prix", "circuit": "Autódromo Hermanos Rodríguez"},
    21: {"name": "São Paulo Grand Prix", "circuit": "Autódromo José Carlos Pace"},
    22: {"name": "Las Vegas Grand Prix", "circuit": "Las Vegas Street Circuit"},
    23: {"name": "Qatar Grand Prix", "circuit": "Lusail International Circuit"},
    24: {"name": "Abu Dhabi Grand Prix", "circuit": "Yas Marina Circuit"}
}

print(f"F1 2025 calendar loaded with {len(F1_2025_CALENDAR)} races")

F1 2025 calendar loaded with 24 races


## 6. F1 Data Retrieval Tools

In [6]:
class F1DataTools:
    """F1 data retrieval using FastF1."""
    
    def get_event_info(self, year: int, round: int) -> Dict[str, Any]:
        """Get event information for a specific race."""
        try:
            event = fastf1.get_event(year, round)
            
            result = {
                "year": year,
                "round": round,
                "event_name": event.EventName,
                "country": event.Country,
                "location": event.Location,
                "official_event_name": event.OfficialEventName,
                "event_date": event.EventDate.isoformat() if hasattr(event.EventDate, 'isoformat') else str(event.EventDate),
                "event_format": event.EventFormat,
            }
            
            print(f"Event info retrieved: {event.EventName}")
            return result
            
        except Exception as e:
            print(f"Error getting event info: {e}")
            return {"error": str(e)}
    
    def get_session_results(self, year: int, round: int, session_type: str = "R") -> Dict[str, Any]:
        """Get session results for a specific race."""
        try:
            session = fastf1.get_session(year, round, session_type)
            session.load()
            
            results = session.results
            
            drivers_results = []
            for idx, row in results.iterrows():
                driver_result = {
                    "position": int(row['Position']) if pd.notna(row['Position']) else None,
                    "driver_number": str(row['DriverNumber']) if pd.notna(row['DriverNumber']) else None,
                    "abbreviation": str(row['Abbreviation']) if pd.notna(row['Abbreviation']) else None,
                    "full_name": str(row['FullName']) if pd.notna(row['FullName']) else None,
                    "team": str(row['TeamName']) if pd.notna(row['TeamName']) else None,
                    "grid_position": int(row['GridPosition']) if pd.notna(row['GridPosition']) else None,
                    "time": str(row['Time']) if pd.notna(row['Time']) else None,
                    "status": str(row['Status']) if pd.notna(row['Status']) else None,
                    "points": float(row['Points']) if pd.notna(row['Points']) else 0.0,
                }
                drivers_results.append(driver_result)
            
            result = {
                "year": year,
                "round": round,
                "session_type": session_type,
                "session_name": session.name,
                "results": drivers_results
            }
            
            print(f"Session results retrieved: {session.name} ({len(drivers_results)} drivers)")
            return result
            
        except Exception as e:
            print(f"Error getting session results: {e}")
            return {"error": str(e)}
    
    def get_driver_info(self, driver: str, year: int) -> Dict[str, Any]:
        """Get driver information."""
        try:
            session = fastf1.get_session(year, 1, 'R')
            session.load()
            
            driver_upper = driver.upper()
            driver_info = None
            
            for idx, row in session.results.iterrows():
                if (str(row['Abbreviation']).upper() == driver_upper or 
                    driver_upper in str(row['FullName']).upper()):
                    driver_info = {
                        "abbreviation": str(row['Abbreviation']),
                        "full_name": str(row['FullName']),
                        "driver_number": str(row['DriverNumber']),
                        "team": str(row['TeamName']),
                        "year": year
                    }
                    break
            
            if driver_info:
                print(f"Driver info retrieved: {driver_info['full_name']}")
                return driver_info
            else:
                return {"error": f"Driver '{driver}' not found in {year} season"}
                
        except Exception as e:
            print(f"Error getting driver info: {e}")
            return {"error": str(e)}

# Initialize tools
f1_tools = F1DataTools()
print("F1DataTools initialized")

F1DataTools initialized


## 7. Test F1 Data Retrieval

In [7]:
# Test with 2024 Bahrain GP
print("Testing F1 data retrieval...")
print("-" * 60)

# Test 1: Get event info
print("\nTest 1: Event Info")
event = f1_tools.get_event_info(2024, 1)
if "error" not in event:
    print(f"  Event: {event['event_name']}")
    print(f"  Location: {event['location']}")
    print(f"  Date: {event['event_date']}")

# Test 2: Get race results
print("\nTest 2: Race Results")
results = f1_tools.get_session_results(2024, 1, "R")
if "error" not in results:
    print(f"  Session: {results['session_name']}")
    print(f"  Top 3:")
    for driver in results['results'][:3]:
        print(f"    {driver['position']}. {driver['full_name']} ({driver['team']})")

# Test 3: Get driver info
print("\nTest 3: Driver Info")
driver = f1_tools.get_driver_info("VER", 2024)
if "error" not in driver:
    print(f"  Driver: {driver['full_name']}")
    print(f"  Number: {driver['driver_number']}")
    print(f"  Team: {driver['team']}")

print("\n" + "-" * 60)
print("All tests completed successfully")

Testing F1 data retrieval...
------------------------------------------------------------

Test 1: Event Info


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Event info retrieved: Bahrain Grand Prix
  Event: Bahrain Grand Prix
  Location: Sakhir
  Date: 2024-03-02T00:00:00

Test 2: Race Results


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Session results retrieved: Race (20 drivers)
  Session: Race
  Top 3:
    1. Max Verstappen (Red Bull Racing)
    2. Sergio Perez (Red Bull Racing)
    3. Carlos Sainz (Ferrari)

Test 3: Driver Info


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']


Driver info retrieved: Max Verstappen
  Driver: Max Verstappen
  Number: 1
  Team: Red Bull Racing

------------------------------------------------------------
All tests completed successfully


## 8. Agent 1: Data Collection


In [ ]:
class DataCollectionAgent:
    """Agent 1: Validates user input and collects comprehensive F1 race data."""
    
    def __init__(self, f1_tools: F1DataTools, calendar: Dict[int, Dict[str, str]]):
        self.f1_tools = f1_tools
        self.calendar = calendar
        self.year = 2025  # Default to 2025 season
    
    def validate_input(self, user_input: str) -> Optional[int]:
        """
        Validate user input and return the round number.
        Accepts either:
        - Round number (e.g., "1", "5", "24")
        - GP name (e.g., "Bahrain", "Monaco Grand Prix")
        
        Returns round number if valid, None otherwise.
        """
        user_input = user_input.strip()
        
        # Try to parse as round number
        try:
            round_num = int(user_input)
            if round_num in self.calendar:
                return round_num
            else:
                print(f"❌ Round {round_num} is not valid. Must be between 1 and {len(self.calendar)}.")
                return None
        except ValueError:
            pass
        
        # Try to match GP name
        user_input_lower = user_input.lower()
        for round_num, info in self.calendar.items():
            gp_name_lower = info['name'].lower()
            circuit_name_lower = info['circuit'].lower()
            
            # Match if user input is contained in GP name or circuit name
            if (user_input_lower in gp_name_lower or 
                gp_name_lower.find(user_input_lower) != -1 or
                user_input_lower in circuit_name_lower):
                return round_num
        
        print(f"❌ '{user_input}' doesn't match any GP in the {self.year} calendar.")
        return None
    
    def collect_race_data(self, round_num: int) -> Optional[Dict[str, Any]]:
        """
        Collect comprehensive race data for a given round.
        
        Returns dictionary with:
        - GP info (name, circuit, length, laps, fastest lap)
        - Starting grid
        - Final race positions
        - Podium finishers with details
        - Key race events (DNFs, penalties)
        """
        print(f"\nCollecting data for Round {round_num}: {self.calendar[round_num]['name']}")
        print("-" * 60)
        
        try:
            # Get event information
            print("Fetching event information...")
            event_info = self.f1_tools.get_event_info(self.year, round_num)
            if "error" in event_info:
                # Try with 2024 data as fallback
                print(f" {self.year} data not available, trying 2024...")
                event_info = self.f1_tools.get_event_info(2024, round_num)
                if "error" in event_info:
                    print(f" Failed to retrieve event info: {event_info['error']}")
                    return None
                self.year = 2024  # Update year for subsequent calls
            
            # Get race session data
            print(" Fetching race results...")
            race_results = self.f1_tools.get_session_results(self.year, round_num, "R")
            if "error" in race_results:
                print(f" Failed to retrieve race results: {race_results['error']}")
                return None
            
            # Load full session for additional data
            session = fastf1.get_session(self.year, round_num, "R")
            session.load()
            
            # Extract circuit info
            try:
                circuit_info = session.get_circuit_info()
                circuit_length = circuit_info.length  # Length in meters
            except (AttributeError, Exception):
                circuit_length = "N/A"
            
            total_laps = session.total_laps if hasattr(session, 'total_laps') else "N/A"
            
            # Get fastest lap
            laps = session.laps
            if not laps.empty:
                fastest_lap = laps.pick_fastest()
                fastest_lap_time = str(fastest_lap['LapTime']) if pd.notna(fastest_lap['LapTime']) else "N/A"
                fastest_lap_driver = fastest_lap['Driver'] if pd.notna(fastest_lap['Driver']) else "N/A"
            else:
                fastest_lap_time = "N/A"
                fastest_lap_driver = "N/A"
            
            # Process results
            results = race_results['results']
            
            # Extract starting grid (sorted by grid position)
            starting_grid = sorted(
                [r for r in results if r['grid_position'] is not None],
                key=lambda x: x['grid_position']
            )
            
            # Extract podium finishers (top 3)
            podium = [r for r in results if r['position'] in [1, 2, 3]]
            podium = sorted(podium, key=lambda x: x['position'])
            
            # Extract DNFs and key events
            dnfs = [r for r in results if r['status'] != 'Finished' and r['status'] != '+1 Lap' and r['status'] != '+2 Laps']
            
            # Identify key overtakes (drivers who gained significant positions)
            position_changes = []
            for r in results:
                if r['grid_position'] is not None and r['position'] is not None:
                    change = r['grid_position'] - r['position']
                    if change != 0:
                        position_changes.append({
                            'driver': r['full_name'],
                            'team': r['team'],
                            'grid': r['grid_position'],
                            'finish': r['position'],
                            'change': change
                        })
            
            # Sort by biggest gainers
            position_changes = sorted(position_changes, key=lambda x: x['change'], reverse=True)
            
            # Compile comprehensive data
            race_data = {
                "race_id": f"{self.year}_R{round_num}",
                "year": self.year,
                "round": round_num,
                "gp_info": {
                    "name": event_info['event_name'],
                    "official_name": event_info.get('official_event_name', event_info['event_name']),
                    "country": event_info['country'],
                    "location": event_info['location'],
                    "circuit": self.calendar[round_num]['circuit'],
                    "date": event_info['event_date'],
                    "circuit_length_km": circuit_length / 1000 if isinstance(circuit_length, (int, float)) else circuit_length,
                    "total_laps": total_laps,
                    "fastest_lap_time": fastest_lap_time,
                    "fastest_lap_driver": fastest_lap_driver
                },
                "starting_grid": [
                    {
                        "position": r['grid_position'],
                        "driver": r['full_name'],
                        "team": r['team'],
                        "driver_number": r['driver_number']
                    }
                    for r in starting_grid[:10]  # Top 10 starters
                ],
                "final_results": [
                    {
                        "position": r['position'],
                        "driver": r['full_name'],
                        "team": r['team'],
                        "time": r['time'],
                        "points": r['points'],
                        "grid_position": r['grid_position']
                    }
                    for r in results if r['position'] is not None
                ],
                "podium": [
                    {
                        "position": r['position'],
                        "driver": r['full_name'],
                        "team": r['team'],
                        "time": r['time'],
                        "points": r['points'],
                        "grid_position": r['grid_position']
                    }
                    for r in podium
                ],
                "key_events": {
                    "dnfs": [
                        {
                            "driver": r['full_name'],
                            "team": r['team'],
                            "status": r['status']
                        }
                        for r in dnfs
                    ],
                    "biggest_gainers": position_changes[:5],  # Top 5 position gainers
                    "biggest_losers": position_changes[-3:] if len(position_changes) > 3 else []  # Bottom 3
                }
            }
            
            print(" Data collection completed successfully!")
            print(f"   - Circuit: {race_data['gp_info']['circuit']}")
            print(f"   - Winner: {race_data['podium'][0]['driver']}")
            print(f"   - DNFs: {len(race_data['key_events']['dnfs'])}")
            print("-" * 60)
            
            return race_data
            
        except Exception as e:
            print(f"Error collecting race data: {e}")
            import traceback
            traceback.print_exc()
            return None
    
    def run(self, user_input: str) -> Optional[Dict[str, Any]]:
        """
        Main execution method for Agent 1.
        Validates input and collects race data.
        """
        # Validate input
        round_num = self.validate_input(user_input)
        if round_num is None:
            return None
        
        # Collect data
        race_data = self.collect_race_data(round_num)
        return race_data

# Initialize Agent 1
agent1 = DataCollectionAgent(f1_tools, F1_2025_CALENDAR)
print("✅ Agent 1 (Data Collection) initialized")


✅ Agent 1 (Data Collection) initialized


## 9. Test Agent 1: Data Collection


In [9]:
print("=" * 70)
print("Testing Agent 1: Input Validation")
print("=" * 70)

# Test 1: Valid round number
print("\n🧪 Test 1: Round number input")
result = agent1.validate_input("1")
print(f"   Input: '1' → Round: {result}")

# Test 2: Valid GP name
print("\n🧪 Test 2: GP name input")
result = agent1.validate_input("Bahrain")
print(f"   Input: 'Bahrain' → Round: {result}")

# Test 3: Partial GP name
print("\n🧪 Test 3: Partial GP name")
result = agent1.validate_input("Monaco")
print(f"   Input: 'Monaco' → Round: {result}")

# Test 4: Invalid input
print("\n🧪 Test 4: Invalid input")
result = agent1.validate_input("Invalid Race")
print(f"   Input: 'Invalid Race' → Round: {result}")

# Test 5: Out of range round
print("\n🧪 Test 5: Out of range round")
result = agent1.validate_input("99")
print(f"   Input: '99' → Round: {result}")

print("\n" + "=" * 70)


Testing Agent 1: Input Validation

🧪 Test 1: Round number input
   Input: '1' → Round: 1

🧪 Test 2: GP name input
   Input: 'Bahrain' → Round: 1

🧪 Test 3: Partial GP name
   Input: 'Monaco' → Round: 8

🧪 Test 4: Invalid input
❌ 'Invalid Race' doesn't match any GP in the 2025 calendar.
   Input: 'Invalid Race' → Round: None

🧪 Test 5: Out of range round
❌ Round 99 is not valid. Must be between 1 and 24.
   Input: '99' → Round: None



In [10]:
print("=" * 70)
print("Testing Agent 1: Full Data Collection")
print("=" * 70)

# Collect data for Bahrain GP (Round 1)
# Note: Using 2024 data since 2025 season hasn't occurred yet
race_data = agent1.run("1")

if race_data:
    print("\n" + "=" * 70)
    print("COLLECTED RACE DATA SUMMARY")
    print("=" * 70)
    
    # GP Information
    print(f"\n📍 GP Information:")
    print(f"   Name: {race_data['gp_info']['name']}")
    print(f"   Circuit: {race_data['gp_info']['circuit']}")
    print(f"   Location: {race_data['gp_info']['location']}, {race_data['gp_info']['country']}")
    print(f"   Date: {race_data['gp_info']['date']}")
    print(f"   Circuit Length: {race_data['gp_info']['circuit_length_km']} km")
    print(f"   Total Laps: {race_data['gp_info']['total_laps']}")
    print(f"   Fastest Lap: {race_data['gp_info']['fastest_lap_time']} ({race_data['gp_info']['fastest_lap_driver']})")
    
    # Starting Grid
    print(f"\n🏁 Starting Grid (Top 5):")
    for i, driver in enumerate(race_data['starting_grid'][:5], 1):
        print(f"   P{driver['position']}: {driver['driver']} ({driver['team']})")
    
    # Podium
    print(f"\n🏆 Podium:")
    for driver in race_data['podium']:
        print(f"   P{driver['position']}: {driver['driver']} ({driver['team']}) - {driver['points']} pts")
        print(f"         Started P{driver['grid_position']}, Time: {driver['time']}")
    
    # Key Events
    print(f"\n⚠️  Key Events:")
    print(f"   DNFs: {len(race_data['key_events']['dnfs'])}")
    for dnf in race_data['key_events']['dnfs']:
        print(f"      - {dnf['driver']} ({dnf['team']}): {dnf['status']}")
    
    # Position Changes
    print(f"\n📈 Biggest Gainers:")
    for driver in race_data['key_events']['biggest_gainers'][:3]:
        print(f"      - {driver['driver']}: P{driver['grid']} → P{driver['finish']} (+{driver['change']} positions)")
    
    # Store in session memory
    store_report(race_data['race_id'], race_data)
    print(f"\n💾 Report stored in session memory with ID: {race_data['race_id']}")
    
    print("\n" + "=" * 70)
else:
    print("\n❌ Failed to collect race data")


core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]


Testing Agent 1: Full Data Collection

------------------------------------------------------------
Fetching event information...
Event info retrieved: Australian Grand Prix
 Fetching race results...


req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api          

Session results retrieved: Race (20 drivers)


core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']


Error collecting race data: 'CircuitInfo' object has no attribute 'total_distance'

❌ Failed to collect race data


Traceback (most recent call last):
  File "C:\Users\Inteli\AppData\Local\Temp\ipykernel_9052\716811519.py", line 85, in collect_race_data
    circuit_length = session.get_circuit_info().total_distance if hasattr(session, 'get_circuit_info') else "N/A"
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'CircuitInfo' object has no attribute 'total_distance'


In [12]:
def interactive_agent1_session():
    """
    Interactive session with Agent 1.
    Continuously prompts for race input until user exits.
    """
    print("=" * 70)
    print("F1 REPORT SYSTEM - AGENT 1 (Data Collection)")
    print("=" * 70)
    print("\nWelcome! I can help you analyze F1 2025 races.")
    print("You can provide:")
    print("  - Round number (e.g., '1', '5', '24')")
    print("  - GP name (e.g., 'Bahrain', 'Monaco', 'Silverstone')")
    print("\nType 'exit' or 'quit' to end the session.\n")
    print("-" * 70)
    
    while True:
        # Prompt user
        user_input = input("\n🏁 Which race from F1 2025 do you want to analyze? ").strip()
        
        # Check for exit commands
        if user_input.lower() in ['exit', 'quit', 'q', '']:
            print("\n👋 Thank you for using the F1 Report System!")
            break
        
        # Run Agent 1
        race_data = agent1.run(user_input)
        
        if race_data:
            # Display summary
            print(f"\n✅ Successfully collected data for {race_data['gp_info']['name']}")
            print(f"   Winner: {race_data['podium'][0]['driver']} ({race_data['podium'][0]['team']})")
            print(f"   Circuit: {race_data['gp_info']['circuit']}")
            print(f"   Race ID: {race_data['race_id']}")
            
            # Store in memory
            store_report(race_data['race_id'], race_data)
            print(f"   📊 Stored in session memory")
        else:
            print("\n⚠️  Please try again with a valid race.")
        
        print("-" * 70)
    
    # Show session summary
    history = get_history()
    if history:
        print(f"\n📋 Session Summary: Analyzed {len(history)} race(s)")
        for entry in history:
            print(f"   - {entry['race_id']} at {entry['timestamp']}")

# Uncomment the line below to run the interactive session
interactive_agent1_session()


F1 REPORT SYSTEM - AGENT 1 (Data Collection)

Welcome! I can help you analyze F1 2025 races.
You can provide:
  - Round number (e.g., '1', '5', '24')
  - GP name (e.g., 'Bahrain', 'Monaco', 'Silverstone')

Type 'exit' or 'quit' to end the session.

----------------------------------------------------------------------

------------------------------------------------------------
Fetching event information...
Event info retrieved: Australian Grand Prix
 Fetching race results...


core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core    

Session results retrieved: Race (20 drivers)


core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']


Error collecting race data: 'CircuitInfo' object has no attribute 'total_distance'

⚠️  Please try again with a valid race.
----------------------------------------------------------------------


Traceback (most recent call last):
  File "C:\Users\Inteli\AppData\Local\Temp\ipykernel_9052\716811519.py", line 85, in collect_race_data
    circuit_length = session.get_circuit_info().total_distance if hasattr(session, 'get_circuit_info') else "N/A"
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'CircuitInfo' object has no attribute 'total_distance'



👋 Thank you for using the F1 Report System!


## 10. Fixed: Test Circuit Info


In [ ]:
print("=" * 70)
print("Testing Fixed Circuit Info - Agent 1")
print("=" * 70)

# Test with Round 1 (Bahrain GP)
race_data = agent1.run("1")

if race_data:
    print("\n✅ SUCCESS! Data collection working properly")
    print("\n📍 GP Information:")
    print(f"   Name: {race_data['gp_info']['name']}")
    print(f"   Circuit: {race_data['gp_info']['circuit']}")
    print(f"   Circuit Length: {race_data['gp_info']['circuit_length_km']} km")
    print(f"   Total Laps: {race_data['gp_info']['total_laps']}")
    print(f"   Fastest Lap: {race_data['gp_info']['fastest_lap_time']} by {race_data['gp_info']['fastest_lap_driver']}")
    
    print("\n🏆 Podium:")
    for driver in race_data['podium']:
        print(f"   P{driver['position']}: {driver['driver']} ({driver['team']})")
    
    print("\n" + "=" * 70)
else:
    print("\n❌ Data collection failed")
